In [ ]:
import re
import random
!pip install pylatex
!apt-get install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-lang-cyrillic texlive-bibtex-extra biber
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from pylatex import UnsafeCommand
from pylatex.base_classes import Environment
from pylatex.utils import NoEscape
from pylatex import Document, Section, Command
from pylatex.utils import NoEscape
from pylatex.package import Package

# import nltk
# nltk.download('punkt')


# device = "cuda" if torch.cuda.is_available() else "cpu"
# # Загрузите модель и токенизатор Mistral
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1").to(device)
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")



# pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ru", device='cuda')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pylatex: filename=PyLaTeX-1.4.2-py3-none-any.whl size=43559 sha256=ad5575e22ef51146921b3d52251ad28e99416d7568d8f655da2978e908e57cdf
  Stored in directory: /root/.cache/pip/wheels/a3/60/09/c6f9f98feac18da1b5fc02bd765c6b3cb9a0f75955a12b27ad
Successfully built pylatex
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-adf-accanthis fonts-adf-berenis fonts-adf-gillius fonts-adf-universalis fonts-cabin
  fonts-cantarell fonts-comfortaa fonts-croscore fonts-crosextra-caladea fonts-crosextra-carlito
  fonts-dejavu-core fonts-dejavu-extra fonts-droid-fallback fonts-ebgaramond fonts-ebgaramond-extra
  fonts-font-awesome fonts-freefont-otf fonts-freefo

In [ ]:
class PromptMaster:
    def __init__(self, model, tokenizer, device="cuda"):
        self.name_prompt = """Create a title for a research paper that will explore your chosen topic. Imagine that your paper will be a major journal article in a relevant field of study, and the title should reflect the key aspects of the research, attracting the attention of potential readers. Please keep in mind that the title should be short but informative and convey the essence of your research as clearly as possible.
Below should be located only 3 topics and 3 titles to them
Topic: Phisics
Title: Developments in Artificial Intelligence in Phisics research
Topic: Medicine
Title: Impact of Climate Change on Marine Ecosystems
Topic: {}
Title:"""
        self.annotation_prompt = """Create an abstract for a scientific article exploring your chosen topic. Imagine that your article will be published in an important scientific journal in the relevant field of research, and the abstract should clearly and briefly reflect the essence of your work, attracting the attention of potential readers. Please note that the abstract should be informative, relevant and contain the main aspects of the study.
Below should be located only 3 topics and 3 annotations to them
Title: Development of artificial intelligence in medical diagnostics
Example of an aмnnotation: This article examines the latest achievements in the field of artificial intelligence and their application in medical diagnostics. The work evaluates the potential of AI technologies for more accurate and rapid diagnoses, which can lead to significant improvements in healthcare.
Title: The impact of climate change on marine ecosystems
Example of an annotation: This article explores the impact of climate change, including changes in temperature and ocean conditions, on marine ecosystems. The authors consider the consequences of climate change for biodiversity and ecological balance in the seas and oceans.
Title: {}
Example of an annotation:"""

        self.keyword_prompt = """Create an abstract for a scientific article and keywords that best describe the content of your scientific work. Imagine that your article will be published in a scientific journal, and the abstract should clearly and briefly reflect the essence of your work, and keywords will help readers quickly understand its context and content. Make sure that the abstract and keywords reflect the main aspects of your research as accurately and accurately as possible.
Below should be located only 3 topics and 3 keywords to them
Abstract: This article examines the impact of climate change on marine ecosystems and biodiversity. The authors analyze how changes in temperature and sea conditions affect marine organisms and ecological balance in the oceans.
Keywords: climate change, marine ecosystems, biodiversity, marine organisms, oceanic ecology.
Abstract: This article examines the use of artificial intelligence in medical diagnostics and its impact on the accuracy of diagnoses and the quality of healthcare. The authors consider the latest achievements in the field of medical diagnostics using AI.
Keywords: artificial intelligence, medical diagnostics, accuracy of diagnoses, healthcare, technologies in medicine.
Abstract: {}
Keywords:"""

        self.introduction_prompt = """Write an introduction and an abstract to your scientific work. The introduction should represent the importance and relevance of the chosen topic and establish the context of the study. The abstract should clearly and concisely describe the key aspects of your work, including the purpose, methodology and main results. Your scientific work will be published in a scientific journal, so the introduction and abstract should be informative and attractive to potential readers. Make paragraphs and divide the text into logical parts.
Topic: {}
Annotation: {}
Keywords: {}
Introduction:"""

        self.Mam_prompt = """Describe the section 'Materials and methods' for your scientific work on the chosen topic, taking into account the introduction. In this section, you must provide information about materials, data about the objects under study, methods of data collection and research. Please note that your scientific work will be published in a scientific journal, so the description of the 'Materials and Methods' should be clear and detailed. Make paragraphs and divide the text into logical parts.

Topic:{}
Introduction:{}
Materials and methods:"""

        self.prompt_result = """Task: Instruct the model to compose the 'Results' section for a scientific article. In this section, you should summarize the outcomes of the study or research conducted on the specified topic. The 'Results' should provide a clear and concise overview of the findings, including any statistical data, experimental observations, or significant outcomes. Make paragraphs and divide the text into logical parts.

Topic: {}
Mam: {}
Summary of Materials and Methods:"""

        self.prompt_conclusion = """Task: generate well-structured and informative conclusions that reflect the significance and implications of the research findings, ensuring that the conclusions are grounded in the provided results, topic, and summary of Materials and Methods, and that they align with the study's objectives. Make paragraphs and divide the text into logical parts.

Topic: {}
Results: {}
Summary of Materials and Methods:"""

        self.prompt_ref  = """Task: Generate a comprehensive list of references related to the topic "Quantum Physics" by leveraging the content of the generated article's abstract and keywords.
Example:
Title: Quantum physics fundamental branch of physics
Abstract: Quantum physics, also known as quantum mechanics, is a fundamental branch of physics that deals with the behavior of particles on a quantum level. It has far-reaching applications in various fields, including quantum computing, quantum cryptography, and quantum optics.
Keywords: Quantum physics, Quantum mechanics, Quantum computing, Quantum cryptography, Quantum optics.
List of References:
1. Albert, David Z. (1992). Quantum Mechanics and Experience. Harvard University Press.
2. Griffiths, David J. (2005). Introduction to Quantum Mechanics. Pearson.
3. Nielsen, Michael A., & Chuang, Isaac L. (2010). Quantum Computation and Quantum Information. Cambridge University Press.
4. Aspect, Alain (1999). Bell's Theorem: The Naive View of an Experimentalist. Springer.
5. Gisin, Nicolas (2005). Quantum Cryptography. Reviews of Modern Physics, 74(1), 145-195.
6. Scully, Marlan O., & Zubairy, M. Suhail (1997). Quantum Optics. Cambridge University Press.
7. Mermin, N. David (2007). Quantum Computer Science: An Introduction. Cambridge University Press.
8. Bruss, Dagmar (2007). Optimal E91 and singlet fraction. Physical Review Letters, 93(21), 210501.
9. Sakurai, J. J., & Napolitano, Jim (2010). Modern Quantum Mechanics. Pearson.
10. Pan, Jian-Wei, et al. (2012). Quantum teleportation between remote atomic-ensemble quantum memories. Nature, 482(7384), 489-494.
Title:{}
Abstract: {}
Keywords: {}
List of References:"""

        self.model=model
        self.tokenizer=tokenizer

    def gen_model(self, prompt, list_toformat):
        input_ids = self.tokenizer.encode(prompt.format(*list_toformat), return_tensors="pt", max_length=1024, truncation=True).to(device)
        output = self.model.generate(input_ids,
                            temperature=0.5,
                            top_p=0.95,
                            repetition_penalty=1.15,
                            max_new_tokens = 512)
        generated_text = self.tokenizer.decode(output[0, input_ids.shape[1]:], skip_special_tokens=True)
        return generated_text

    def gen_art(self, topic):
        self.name = self.gen_model(self.name_prompt, [topic])
        self.annotation = self.gen_model(self.annotation_prompt, [self.name])
        self.keyword = self.gen_model(self.keyword_prompt, [self.annotation])
        self.introduction = self.gen_model(self.introduction_prompt, [topic, self.annotation, self.keyword])
        self.Mam = self.gen_model(self.Mam_prompt, [self.name, self.introduction])
        self.result = self.gen_model(self.prompt_result, [self.name, self.Mam])
        self.conclusion = self.gen_model(self.prompt_conclusion, [self.name, self.result])
        self.ref = self.gen_model(self.prompt_ref, [self.name, self.annotation, self.keyword])
        #print(self.prompt_ref.format(*[name, annotation, keyword]))
        # result_text = 'Name' + '\n' + pipe(name)[0]['translation_text']  + '\n\n'
        # res_dict = {'annotation': annotation,
        #              'keyword': keyword, 'introduction': introduction,
        #              'Mam': Mam, 'result': result,
        #              'conclusion': conclusion}
        # for part in res_dict:
        #     result_text += part + '\n' + '. '.join([pipe(i)[0]['translation_text'] for i in res_dict[part].split('.')[:-1]]) + '\n\n'



        # result_text = result_text.replace('..', '.')
        # result_text = re.sub(r'&\w+;', '', result_text)
        # result_text = re.sub(r' +', ' ', result_text)

        # result_text += 'ref' + ref

        # return result_text

#     def split_into_paragraphs(self, text):
#         num = random.randint(3, 5)
#         sentences = nltk.sent_tokenize(text)
#         paragraphs = [sentences[i:i+sentences_per_paragraph] for i in range(0, len(sentences), num)]
#         return [" ".join(paragraph) for paragraph in paragraphs]




In [ ]:
class MultiColumnEnvironment(Environment):

    _latex_name = 'MultiColumn'
    packages = [Package('multicol')]

def add_preambule(doc, name):
    doc.preamble.append(Command('title', name))
    doc.preamble.append(Command('author', 'ikanam'))
    doc.preamble.append(Command('date', NoEscape(r'\Large \today')))
    doc.append(NoEscape(r'\maketitle'))

def make_abstract(doc, annotation, key_words):
    doc.append(NoEscape(r'\begin{abstract}'))
    doc.append(NoEscape(r'{\Large ' + annotation + r' \\}'))
    doc.append(NoEscape(f'\par \Large Ключевые слова: \Large {key_words};'))
    doc.append(NoEscape(r'\end{abstract}'))

def simple_section(doc, name, text, cite=False):
    with doc.create(Section(name)):
        doc.append(text)
        if cite:
            doc.append(Command('cite', arguments=[r'carno']))

def generate_scientific_article(data, path):
    """Генерирование оригинальной научной статьи с помощью загруженной модели

    Args:
        theme (str): Название нашей темы (1 из 50)


    Схема самой статьи:

    1. Название статьи
    2. Аннотация
    3. Ключевые слова
    4. Введение
    5. Материалы и методы
    6. Результаты
    7. Выводы
    8. Список литературы

    TODO: Схема генерации промптов
    Схема генерации промптов - нужно прописать, т.е например
    нужно сначала генерировать полный текст и только потом ключевые слова.
    """
    geometry_options = {"top":"3.5cm","bottom":"3.5cm", "left":"3.7cm",\
        "right":"4.7cm", "columnsep":"30pt"}
    doc = Document(geometry_options=geometry_options)
    doc.append(NoEscape(r'\fontsize{15}{15pt}\selectfont'))
    new_env = UnsafeCommand('newenvironment', 'MultiColumn',
                        extra_arguments=[r'\begin{multicols}{2}', r'\end{multicols}'])
    doc.append(new_env)

    doc.packages.append(Package("fontenc", options=["T2A"]))
    doc.packages.append(Package("inputenc", options=["utf8"]))
    doc.packages.append(Package("babel", options=["russian"]))
    doc.preamble.append(NoEscape(r'\newcommand{\mytitle}[1]{\title{\Huge #1}}'))
    add_preambule(doc, data['name'])
    make_abstract(doc, data['annotation'], data['keyword'])
    with doc.create(MultiColumnEnvironment()) as environment:
        simple_section(environment, 'Введение', data['introduction'], True)
        simple_section(environment, 'Материалы и методы', data['mam'], True)
        simple_section(environment, 'Результаты', data['result'], True)
        simple_section(environment, 'Выводы', data['conclusion'], True)
        simple_section(environment, 'Список литературы', data['ref'], False)



    doc.generate_pdf(path, clean_tex=False)


In [ ]:
import os
import json

extensions_to_remove = ['.aux', '.log', '.tex']
data_folder = "/content/data/data"
saving_folder = '/content/articles/{}'
saving_pdf_path = '/content/articles/{}/{}'
# Step 2: Loop through the files in the data folder
for filename in os.listdir(data_folder):
    if filename.endswith(".json"):
        # Step 3: Extract the first and second IDs
        first_id, second_id = filename[:-5].split("_")
        # Step 4: Check if a folder with the second ID exists. If not, create it.
        destination_folder = os.path.join(data_folder, filename)
        with open(destination_folder, encoding='utf-8') as f:
          data = json.load(f)

        if not os.path.exists(saving_folder.format(second_id)):
            os.makedirs(saving_folder.format(second_id))

        try:
          generate_scientific_article(data, saving_pdf_path.format(second_id, filename[:-5]))
        except:
          pass

        for filename in os.listdir(saving_folder.format(second_id)):
          if filename.endswith(tuple(extensions_to_remove)):
              file_path = os.path.join(saving_folder.format(second_id), filename)
              os.remove(file_path)




/content/data/data/3_7.json 3_7.json
/content/data/data/3_31.json 3_31.json
/content/data/data/1_26.json 1_26.json
/content/data/data/1_4.json 1_4.json
/content/data/data/2_9.json 2_9.json
/content/data/data/1_1.json 1_1.json
/content/data/data/1_30.json 1_30.json
/content/data/data/1_3.json 1_3.json
/content/data/data/3_6.json 3_6.json
/content/data/data/2_8.json 2_8.json
/content/data/data/1_9.json 1_9.json
/content/data/data/2_4.json 2_4.json
/content/data/data/2_31.json 2_31.json
/content/data/data/3_27.json 3_27.json
/content/data/data/1_32.json 1_32.json
/content/data/data/1_5.json 1_5.json
/content/data/data/2_29.json 2_29.json
/content/data/data/2_7.json 2_7.json
/content/data/data/3_32.json 3_32.json
/content/data/data/2_10.json 2_10.json
/content/data/data/2_6.json 2_6.json
/content/data/data/3_3.json 3_3.json
/content/data/data/3_4.json 3_4.json
/content/data/data/1_27.json 1_27.json
/content/data/data/1_8.json 1_8.json
/content/data/data/2_26.json 2_26.json
/content/data/da

In [88]:
!rm -rf data